In [ ]:
import pandas as pd



- To forecast the consumption for each quarter of an hour over the next 24 hours (96 quarters).
- To forecast the consumption for each hour over the next four days (96 hours).
- To forecast the consumption for each day over the coming month (30 days).


# Historical Consumption
A selected time series of consumption data for over 200 buildings.

- obs_id - An arbitrary ID for the observationaa
- SiteId - An arbitrary ID number for the building, matches across datasets
- ForecastId - An ID for a timeseries that is part of a forecast (can be matched with the submission file)
- Timestamp - The time of the measurement
- Value - A measure of consumption for that building

# Building Metadata
Additional information about the included buildings.

- SiteId - An arbitrary ID number for the building, matches across datasets
- Surface - The surface area of the building
- BaseTemperature - The base temperature for the building
- [DAY_OF_WEEK]IsDayOff - True if DAY_OF_WEEK is not a work day

# Historical Weather Data
This dataset contains temperature data from several stations near each site. For each site several temperature measurements were retrieved from stations in a radius of 30 km if available.

Note: Not all sites will have available weather data.

Note: Weather data is available for test periods under the assumption that reasonably accurate forecasts will be available to algorithms that the time that we are attempting to make predictions about the future.

- SiteId - An arbitrary ID number for the building, matches across datasets
- Timestamp - The time of the measurement
- Temperature - The temperature as measured at the weather station
- Distance - The distance in km from the weather station to the building in km

# Public Holidays
Public holidays at the sites included in the dataset, which may be helpful for identifying days where consumption may be lower than expected.

Note: Not all sites will have available public holiday data.

- SiteId - An arbitrary ID number for the building, matches across datasets
- Date - The date of the holiday
- Holiday - The name of the holiday


In [7]:
train.head()

,obs_id,SiteId,Timestamp,ForecastId,Value
0,744519,1,2014-09-03 00:00:00,1,9.096555e+05
1,7627564,1,2014-09-04 00:00:00,1,1.748273e+06
2,7034705,1,2014-09-05 00:00:00,1,NaN
3,5995486,1,2014-09-06 00:00:00,1,NaN
4,7326510,1,2014-09-07 00:00:00,1,NaN


In [17]:
train = pd.read_csv('input/train.csv')
metadata = pd.read_csv('input/metadata.csv')
#weather = pd.read_csv('input/weather.csv')
#holidays = pd.read_csv('input/holidays.csv', nrows=10)
#subm_fmt = pd.read_csv('input/submission_format.csv', nrows=10)
#subm_freq = pd.read_csv('input/submission_frequency.csv', nrows=10)

In [18]:
train.head()

,obs_id,SiteId,Timestamp,ForecastId,Value
0,744519,1,2014-09-03 00:00:00,1,9.096555e+05
1,7627564,1,2014-09-04 00:00:00,1,1.748273e+06
2,7034705,1,2014-09-05 00:00:00,1,NaN
3,5995486,1,2014-09-06 00:00:00,1,NaN
4,7326510,1,2014-09-07 00:00:00,1,NaN


In [22]:
len(train), len(train.SiteId.unique()), len(train.ForecastId.unique())

(6559830, 267, 6974)

In [23]:
train.Value.describe()

count    6.473229e+06
mean     3.560104e+05
std      3.309454e+08
min      0.000000e+00
25%      3.492465e+03
50%      1.408683e+04
75%      4.649371e+04
max      6.404683e+11
Name: Value, dtype: float64